<a href="https://colab.research.google.com/github/stefanziog/Quantized-Deep-Neural-Networks/blob/main/aware/tf_flower_aware.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
! pip uninstall -y tensorflow
! pip install -q tf-nightly
! pip install -q tensorflow-model-optimization

In [ ]:
import tempfile
import os

import tensorflow as tf
import numpy as np
import tensorflow_model_optimization as tfmot

import pathlib
import PIL

from tensorflow import keras

In [ ]:
# load train and test dataset
def load_dataset():
# load dataset
  dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
  data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
  data_dir = pathlib.Path(data_dir)

  image_count = len(list(data_dir.glob('*/*.jpg')))
  print(image_count)
  roses = list(data_dir.glob('roses/*'))
  PIL.Image.open(str(roses[0]))

  categories = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]



  daisy = list(data_dir.glob('daisy/*'))
  dandelion = list(data_dir.glob('dandelion/*'))
  roses = list(data_dir.glob('roses/*'))
  sunflowers = list(data_dir.glob('sunflowers/*'))
  tulips = list(data_dir.glob('turlips/*'))

  data = []
  labels = []

  for i in daisy:
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb',
    target_size =  (280,280))
    image=np.array(image)
    data.append(image)
    labels.append(0)
  for i in dandelion:
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb',
    target_size= (280,280))
    image=np.array(image)
    data.append(image)
    labels.append(1)
  for i in roses:
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb',
    target_size= (280,280))
    image=np.array(image)
    data.append(image)
    labels.append(2)
  for i in sunflowers:
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb',
    target_size= (280,280))
    image=np.array(image)
    data.append(image)
    labels.append(3)
  for i in tulips:
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb',
    target_size= (280,280))
    image=np.array(image)
    data.append(image)
    labels.append(3)
  data = np.array(data)
  labels = np.array(labels)

  from sklearn.model_selection import train_test_split
  train_images, test_images, train_labels, test_labels = train_test_split(data, labels, test_size=0.2,
                                                random_state=42)
	
  return train_images, train_labels, test_images, test_labels
# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

In [ ]:
def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy

In [ ]:
# load dataset
train_images, train_labels, test_images, test_labels = load_dataset()

# prepare pixel data
train_images, test_images = prep_pixels(train_images, test_images)

#load model
model = tf.keras.models.load_model('/content/gdrive/MyDrive/Διπλωματική Κώδικες τελικοί/saved_models/tf_flower.h5')

3670


In [ ]:
#Define Quantization Model

quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

q_aware_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantize_layer (QuantizeLay  (None, 280, 280, 3)       3         
er)                                                              
_________________________________________________________________
quant_conv2d (QuantizeWrapp  (None, 140, 140, 64)      9539      
er)                                                              
_________________________________________________________________
quant_max_pooling2d (Quanti  (None, 47, 70, 64)        1         
zeWrapper)                                                       
_________________________________________________________________
quant_conv2d_1 (QuantizeWra  (None, 24, 35, 128)       401667    
pper)                                                            
_________________________________________________________________
quant_max_pooling2d_1 (Quan  (None, 8, 18, 128)        1

In [ ]:
#train and evaluate model against baseline
train_images_subset = train_images[0:1000]  # out of 60000
train_labels_subset = train_labels[0:1000]

q_aware_model.fit(train_images_subset, train_labels_subset,
                  batch_size=500, epochs=5, validation_split=0.1)

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


2/2 [==============================] - 97s 42s/step - loss: 0.1162 - accuracy: 0.9644 - val_loss: 0.0194 - val_accuracy: 1.0000
Epoch 2/5
2/2 [==============================] - 94s 43s/step - loss: 0.0372 - accuracy: 0.9889 - val_loss: 0.1178 - val_accuracy: 0.9500
Epoch 3/5
2/2 [==============================] - 93s 43s/step - loss: 0.0439 - accuracy: 0.9900 - val_loss: 0.0483 - val_accuracy: 0.9900
Epoch 4/5
2/2 [==============================] - 92s 42s/step - loss: 0.0162 - accuracy: 0.9967 - val_loss: 0.1070 - val_accuracy: 0.9500
Epoch 5/5
2/2 [==============================] - 89s 40s/step - loss: 0.0209 - accuracy: 0.9967 - val_loss: 0.0618 - val_accuracy: 0.9800


In [ ]:
#Create Quantize model for TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpq5tt_75_/assets


INFO:tensorflow:Assets written to: /tmp/tmpq5tt_75_/assets


In [ ]:
#Evaluate the quantized model
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

_, q_aware_model_accuracy = q_aware_model.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Quant test accuracy:', q_aware_model_accuracy)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


Baseline test accuracy: 0.7686956524848938
Quant test accuracy: 0.7686956524848938


In [ ]:
#Check the size of the models
# Create float TFLite model.
float_converter = tf.lite.TFLiteConverter.from_keras_model(model)
float_tflite_model = float_converter.convert()

# Measure sizes of models.
_, float_file = tempfile.mkstemp('.tflite')
_, quant_file = tempfile.mkstemp('.tflite')

with open(quant_file, 'wb') as f:
  f.write(quantized_tflite_model)

with open(float_file, 'wb') as f:
  f.write(float_tflite_model)

print("Float model in Mb:", os.path.getsize(float_file) / float(2**20))
print("Quantized model in Mb:", os.path.getsize(quant_file) / float(2**20))

INFO:tensorflow:Assets written to: /tmp/tmp80garp9o/assets


INFO:tensorflow:Assets written to: /tmp/tmp80garp9o/assets


Float model in Mb: 10.27231216430664
Quantized model in Mb: 2.5949020385742188
